<a href="https://colab.research.google.com/github/coltongerth/raster-max-green-comp/blob/main/raster_max_green_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# !pip install raster_tools
!pip install rasterio
!pip install gdown

In [11]:
import rasterio
from rasterio import shutil as rio_shutil
from rasterio.enums import Resampling
from rasterio.windows import Window
import numpy as np
import gdown, zipfile


In [12]:
# Change this to the zipfile containing your raster and mask you desire.
url='https://drive.google.com/file/d/19WqU5Ka9HL1KT2GhzjNb5mUuemSVpFx1/view?usp=sharing'
outfl= r'./raster_data.zip'
gdown.download(url=url,output=outfl,quiet=False,fuzzy=True)
with zipfile.ZipFile(outfl, 'r') as zip_ref:
    zip_ref.extractall(".")

Downloading...
From (original): https://drive.google.com/uc?id=19WqU5Ka9HL1KT2GhzjNb5mUuemSVpFx1
From (redirected): https://drive.google.com/uc?id=19WqU5Ka9HL1KT2GhzjNb5mUuemSVpFx1&confirm=t&uuid=d67380b6-3a89-4e9f-af27-34dffdfec325
To: /content/raster_data.zip
100%|██████████| 111M/111M [00:02<00:00, 52.9MB/s]


In [13]:
# Update the ingrid and mask_path to the names of the rasters desired.
outdir = "."
ingrid = "/content/MaDoyCutwgs.tif"
jday = 167
mask_path = "/content/EvtCutwgs.tif"
year = 521

In [14]:
nodata_value = -9999  # Define a no data value

# First attempt at using John's stuff, not quite there yet.
# ingrid_raster = Raster(ingrid)
# mask_vector = Raster(mask_path)
# print(mask_vector.mask)
# ingrid_raster = clipping.clip(mask_vector.mask,ingrid_raster)
# print(ingrid_raster)



# Open the mask and input grid
with rasterio.open(mask_path) as mask_ds, rasterio.open(ingrid) as ingrid_ds:
    # Prepare output meta from mask
    output_meta = mask_ds.meta.copy()
    output_meta.update({
        'compress': 'lzw',
        'nodata': nodata_value,
        'dtype': 'int32'  # Change data type to signed 32-bit integer
    })

    # Prepare output files
    with rasterio.open(outdir + "/DOYLessJday.tif", "w", **output_meta) as dst_less, \
         rasterio.open(outdir + "/DOYDevclass.tif", "w", **output_meta) as dst_class, \
         rasterio.open(outdir + "/DOYDevclassno.tif", "w", **output_meta) as dst_classno:

        # Ensure block shapes match
        assert mask_ds.block_shapes[0] == ingrid_ds.block_shapes[0]

        # Process each block of data
        for ji, window in mask_ds.block_windows(1):
            # Read a window of the mask
            mask = mask_ds.read(1, window=window)
            # Read the corresponding window of the input grid
            input_grid = ingrid_ds.read(1, window=window)

            # Ensure shapes match exactly
            if mask.shape != input_grid.shape:
                min_shape = (min(mask.shape[0], input_grid.shape[0]), min(mask.shape[1], input_grid.shape[1]))
                mask = mask[:min_shape[0], :min_shape[1]]
                input_grid = input_grid[:min_shape[0], :min_shape[1]]

            # Adjust the input grid
            input_grid = input_grid - jday

            # Calculate DOYLessJday
            DOYLessJday = np.where(mask > 0, input_grid, nodata_value)

            # Calculate DOYDevclass
            DOYDevclass = np.full(DOYLessJday.shape, nodata_value, dtype=np.int32)
            DOYDevclass[(DOYLessJday < -365)] = nodata_value
            DOYDevclass[(DOYLessJday >= -200) & (DOYLessJday <= -35)] = 1
            DOYDevclass[(DOYLessJday > -35) & (DOYLessJday <= -14)] = 2
            DOYDevclass[(DOYLessJday > -14) & (DOYLessJday <= 0)] = 3
            DOYDevclass[(DOYLessJday > 0) & (DOYLessJday <= 14)] = 4
            DOYDevclass[(DOYLessJday > 14) & (DOYLessJday <= 35)] = 5
            DOYDevclass[(DOYLessJday > 35) & (DOYLessJday <= 200)] = 6

            # Calculate DOYDevclassno
            DOYDevclassno = np.where(DOYLessJday == nodata_value, nodata_value, DOYDevclass)

            # Write each block back to the output rasters
            dst_less.write(DOYLessJday, 1, window=window)
            dst_class.write(DOYDevclass, 1, window=window)
            dst_classno.write(DOYDevclassno, 1, window=window)